In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Download TorchVision repo to use some files from
# references/detection
!pip install pycocotools --quiet
!git clone https://github.com/pytorch/vision.git
!git checkout v0.3.0

!cp vision/references/detection/utils.py ./
!cp vision/references/detection/transforms.py ./
!cp vision/references/detection/coco_eval.py ./
!cp vision/references/detection/engine.py ./
!cp vision/references/detection/coco_utils.py ./

Cloning into 'vision'...
remote: Enumerating objects: 531472, done.
remote: Counting objects: 100% (45082/45082), done.
remote: Compressing objects: 100% (2074/2074), done.
remote: Total 531472 (delta 43039), reused 44915 (delta 42935), pack-reused 486390
Receiving objects: 100% (531472/531472), 1.00 GiB | 24.28 MiB/s, done.
Resolving deltas: 100% (496283/496283), done.
fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [3]:
# Basic python and ML Libraries
import os
import random
import numpy as np
import pandas as pd
# for ignoring warnings
import warnings
warnings.filterwarnings('ignore')

# We will be reading images using OpenCV
import cv2

# xml library for parsing xml files
from xml.etree import ElementTree as et

# matplotlib for visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# torchvision libraries
import torch
from torch.utils.data import Subset
import torchvision
from torchvision import transforms as torchtrans  
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# these are the helper libraries imported.
from engine import train_one_epoch, evaluate
import utils
import transforms as T

# for image augmentations
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2


In [4]:
mkdir -p /kaggle/working/test-inriaperson && cp -r /kaggle/input/inriaperson/Test/Annotations/* /kaggle/working/test-inriaperson

In [5]:
cp -r /kaggle/input/inriaperson/Test/JPEGImages/* /kaggle/working/test-inriaperson

In [6]:
# defining the files directory and testing directory
train_dir = '/kaggle/input/kitti-train-test-selection/Train_Test_Selection/Train'
test_dir = '/kaggle/working/test-inriaperson'

In [7]:
#for xml-annotatation dataset
class ImagesDataset(torch.utils.data.Dataset):

    def __init__(self, files_dir, width, height, transforms=None):
        self.transforms = transforms
        self.files_dir = files_dir
        self.height = height
        self.width = width
            
        self.imgs = [image for image in sorted(os.listdir(files_dir)) if image.endswith(('.png', '.jpg'))]
        
        
        # classes: 0 index is reserved for background
        self.classes = ['background','Pedestrian']

    def __getitem__(self, idx):

        img_name = self.imgs[idx]
        image_path = os.path.join(self.files_dir, img_name)

        # reading the images and converting them to correct size and color    
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img_res = cv2.resize(img_rgb, (self.width, self.height), cv2.INTER_AREA)
        # diving by 255
        img_res /= 255.0
        
        # annotation file
        annot_filename = img_name[:-4] + '.xml'
        annot_file_path = os.path.join(self.files_dir, annot_filename)
        
        boxes = []
        labels = []
        tree = et.parse(annot_file_path)
        root = tree.getroot()
        
        # cv2 image gives size as height x width
        wt = img.shape[1]
        ht = img.shape[0]
        
        # box coordinates for xml files are extracted and corrected for image size given
        for member in root.findall('object'):
            label_text = member.find('name').text.lower()
            if label_text in ['ped', 'pedestrian','person']:
                labels.append(self.classes.index('Pedestrian')) 
            
            # bounding box
            xmin = int(member.find('bndbox').find('xmin').text)
            xmax = int(member.find('bndbox').find('xmax').text)
            
            ymin = int(member.find('bndbox').find('ymin').text)
            ymax = int(member.find('bndbox').find('ymax').text)
            
            
            xmin_corr = (xmin/wt)*self.width
            xmax_corr = (xmax/wt)*self.width
            ymin_corr = (ymin/ht)*self.height
            ymax_corr = (ymax/ht)*self.height
            
            boxes.append([xmin_corr, ymin_corr, xmax_corr, ymax_corr])
          
        
        # convert boxes into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        
        
        # getting the areas of the boxes
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        # suppose all instances are not crowd
        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)
        
        labels = torch.as_tensor(labels, dtype=torch.int64)


        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["area"] = area
        target["iscrowd"] = iscrowd
        # image_id
        image_id = idx
        target["image_id"] = image_id
        
  


        if self.transforms:
            
            sample = self.transforms(image = img_res,
                                     bboxes = target['boxes'],
                                     labels = labels)
            
            img_res = sample['image']
            target['boxes'] = torch.Tensor(sample['bboxes'])
            
            
            
        return img_res, target

    def __len__(self):
        return len(self.imgs)

In [8]:
#for txt-annotatation dataset
class KittiDataset(torch.utils.data.Dataset):
    def __init__(self, files_dir, width, height, transforms=None):
        self.transforms = transforms
        self.files_dir = files_dir
        self.height = height
        self.width = width
        self.imgs = [image for image in sorted(os.listdir(files_dir)) if image.endswith(('.png', '.jpg'))]
        self.classes = ['background', 'Pedestrian']

    def __getitem__(self, idx):
        img_name = self.imgs[idx]
        image_path = os.path.join(self.files_dir, img_name)

        # Reading the images and converting them to correct size and color
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img_res = cv2.resize(img_rgb, (self.width, self.height), cv2.INTER_AREA)
        img_res /= 255.0

        # Annotation file
        annot_filename = img_name[:-4] + '.txt'
        annot_file_path = os.path.join(self.files_dir, annot_filename)

        boxes = []
        labels = []

           # cv2 image gives size as height x width
        wt = img.shape[1]
        ht = img.shape[0]
        # Read annotation data from text file
        with open(annot_file_path, 'r') as file:
            for line in file:
                parts = line.strip().split(' ')
                label_text = parts[0].lower()

                if label_text in ['pedestrian', 'person']:
                    labels.append(self.classes.index('Pedestrian'))

                    # Bounding box coordinates
                    xmin = float(parts[4])
                    ymin = float(parts[5])
                    xmax = float(parts[6])
                    ymax = float(parts[7])

                    # Normalize coordinates
                    xmin_corr = (xmin/wt)*self.width
                    xmax_corr = (xmax/wt)*self.width
                    ymin_corr = (ymin/ht)*self.height
                    ymax_corr = (ymax/ht)*self.height

                    boxes.append([xmin_corr, ymin_corr, xmax_corr, ymax_corr])

        # Convert boxes into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        #print("boxes shape: ", boxes.size(), "at index",idx, img_name)
       
      
           

        # Calculate areas of the boxes
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        # Suppose all instances are not crowd
        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)

        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["area"] = area
        target["iscrowd"] = iscrowd

        # image_id
        image_id = torch.tensor([idx])
        target["image_id"] = image_id

        if self.transforms:
            sample = self.transforms(image=img_res, bboxes=target['boxes'], labels=labels)
            img_res = sample['image']
            target['boxes'] = torch.Tensor(sample['bboxes'])
        return img_res, target
    
    

    def __len__(self):
        return len(self.imgs)


In [9]:
# # check train dataset
train_dataset = KittiDataset(train_dir, 1920, 1080)
print('length of dataset = ', len(train_dataset), '\n')

# getting the image and target for a test index.  Feel free to change the index.
img, target = train_dataset[7]
print(img.shape, '\n',target)


length of dataset =  300 

(1080, 1920, 3) 
 {'boxes': tensor([[1069.0474,  486.3744, 1083.4706,  571.7088]]), 'labels': tensor([1]), 'area': tensor([1230.7968]), 'iscrowd': tensor([0]), 'image_id': tensor([7])}


In [10]:
train_dataset.classes

['background', 'Pedestrian']

In [11]:
# check test dataset
test_dataset = ImagesDataset(test_dir, 1920, 1080)
print('length of dataset = ', len(test_dataset), '\n')

# getting the image and target for a test index.  Feel free to change the index.
img, target = test_dataset[7]
print(img.shape, '\n',target)

length of dataset =  288 

(1080, 1920, 3) 
 {'boxes': tensor([[1026.0803,  244.1250, 1339.9618,  894.3750],
        [1703.4034,  196.8750, 1789.6749,  419.6250],
        [1795.1816,  203.6250, 1872.2754,  380.2500]]), 'labels': tensor([1, 1, 1]), 'area': tensor([204101.4219,  19216.9727,  13616.6836]), 'iscrowd': tensor([0, 0, 0]), 'image_id': 7}


In [12]:
test_dataset.classes

['background', 'Pedestrian']

In [13]:

def get_object_detection_model(num_classes):

    # load a model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) 

    return model

In [14]:
# Send train=True fro training transforms and False for val/test transforms
def get_transform(train):
    
    if train:
        return A.Compose([
            A.HorizontalFlip(0.5),
            A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5),
            A.Rotate(limit=15, p=0.5),
            ToTensorV2(p=1.0)
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))
    else:
        return A.Compose([
                            ToTensorV2(p=1.0)
                        ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [15]:
#Adjust the length of test dataset to align with other test datasets
# Define the total number of samples in your dataset
total_samples = len(test_dataset)

# Define the percentage of the dataset you want to load
portion_to_load = 0.3  # Example: load 50% of the dataset

# Calculate the number of samples to load
num_samples_to_load = int(total_samples * portion_to_load)

# Generate random indices to select a random subset of the dataset

indices = random.sample(range(total_samples), num_samples_to_load)

# Create a Subset dataset containing only the selected indices
subset = Subset(test_dataset, indices)
len(subset)


86

In [16]:
# use our dataset and defined transformations
train_dataset = KittiDataset(train_dir, 1920, 1080, transforms= get_transform(train=True))
test_dataset = ImagesDataset(test_dir, 1920, 1080, transforms= get_transform(train=False))
subset_dataset = Subset(test_dataset, indices)
test_dataset = subset_dataset

# define training and validation data loaders
data_loader_train = torch.utils.data.DataLoader(
    train_dataset, batch_size=4, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    test_dataset, batch_size=4, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

In [17]:
# to train on gpu if selected.
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


num_classes = 2

# get the model using our helper function
model = get_object_detection_model(num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:02<00:00, 82.5MB/s]


In [18]:
# training for 10 epochs
num_epochs = 10
for epoch in range(num_epochs):
    # training for one epoch
    train_one_epoch(model, optimizer, data_loader_train, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

Epoch: [0]  [ 0/75]  eta: 0:04:59  lr: 0.000073  loss: 0.7100 (0.7100)  loss_classifier: 0.5627 (0.5627)  loss_box_reg: 0.0757 (0.0757)  loss_objectness: 0.0533 (0.0533)  loss_rpn_box_reg: 0.0183 (0.0183)  time: 3.9959  data: 1.3160  max mem: 4799
Epoch: [0]  [10/75]  eta: 0:01:01  lr: 0.000748  loss: 0.6571 (0.6319)  loss_classifier: 0.3762 (0.3662)  loss_box_reg: 0.1533 (0.1423)  loss_objectness: 0.0345 (0.0790)  loss_rpn_box_reg: 0.0266 (0.0444)  time: 0.9435  data: 0.1455  max mem: 4957
Epoch: [0]  [20/75]  eta: 0:00:43  lr: 0.001423  loss: 0.5011 (0.6169)  loss_classifier: 0.2148 (0.2932)  loss_box_reg: 0.1446 (0.1594)  loss_objectness: 0.0408 (0.1027)  loss_rpn_box_reg: 0.0254 (0.0616)  time: 0.6396  data: 0.0301  max mem: 4958
Epoch: [0]  [30/75]  eta: 0:00:33  lr: 0.002098  loss: 0.2945 (0.5220)  loss_classifier: 0.1024 (0.2322)  loss_box_reg: 0.1192 (0.1493)  loss_objectness: 0.0650 (0.0916)  loss_rpn_box_reg: 0.0197 (0.0489)  time: 0.6470  data: 0.0330  max mem: 4958
Epoch: [

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

creating index...
index created!


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Test:  [ 0/22]  eta: 0:00:21  model_time: 0.3093 (0.3093)  evaluator_time: 0.0169 (0.0169)  time: 0.9788  data: 0.5917  max mem: 4958


...

creating index...
index created!


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Test:  [ 0/22]  eta: 0:00:21  model_time: 0.2985 (0.2985)  evaluator_time: 0.0172 (0.0172)  time: 0.9568  data: 0.5927  max mem: 4958


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

Test:  [21/22]  eta: 0:00:00  model_time: 0.2538 (0.2515)  evaluator_time: 0.0049 (0.0092)  time: 0.3139  data: 0.0288  max mem: 4958
Test: Total time: 0:00:07 (0.3466 s / it)
Averaged stats: model_time: 0.2538 (0.2515)  evaluator_time: 0.0049 (0.0092)
Accumulating evaluation results...
DONE (t=0.02s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.369
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.896
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.170
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.201
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.371
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.235
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.466
 Average Recall     (AR) @[ IoU=0.50:0